In [2]:

import torch
import jieba
import pandas as pd

data_dir = 'https://mirror.coggle.club/dataset/coggle-competition/'
train_data = pd.read_csv(data_dir + 'intent-classify/train.csv', sep='\t', header=None)
test_data = pd.read_csv(data_dir + 'intent-classify/test.csv', sep='\t', header=None)

In [3]:
train_data.head()

,0,1
0,还有双鸭山到淮阴的汽车票吗13号的,Travel-Query
1,从这里怎么回家,Travel-Query
2,随便播放一首专辑阁楼里的佛里的歌,Music-Play
3,给看一下墓王之王嘛,FilmTele-Play
4,我想看挑战两把s686打突变团竞的游戏视频,Video-Play


In [4]:
train_data = train_data.sample(frac=1.0)

In [5]:
# Pandas.factorize()是Pandas模块提供的一种数据编码方式，它将每个类别数据转换成一个整数(从0开始)，有助于进行数据分析和建模。
train_data[1], lbl = pd.factorize(train_data[1])

print(train_data[1].head(10))  # 已经转换成了整数
print(lbl)  # 去重后的标签列表

11226    0
8696     1
8308     2
6112     3
1434     1
4153     1
11804    4
9178     5
11057    6
10213    3
Name: 1, dtype: int64
Index(['Audio-Play', 'Travel-Query', 'Calendar-Query', 'Video-Play',
       'HomeAppliance-Control', 'Alarm-Update', 'Weather-Query',
       'Radio-Listen', 'Music-Play', 'FilmTele-Play', 'Other',
       'TVProgram-Play'],
      dtype='object')


In [43]:
def coustom_data_iter(texts, labels):
    for x, y in zip(texts, labels):
        yield x, y

In [44]:
train_iter = coustom_data_iter(train_data[0].values[:], train_data[1].values[:])

train_iter

<generator object coustom_data_iter at 0x000001703A793200>

In [13]:

# !pip install torchtext

# 相关参考
# https://www.cnblogs.com/Fortunater/p/16971419.html

# 构建词典
from torchtext.vocab import build_vocab_from_iterator

tokenizer = jieba.lcut

def yield_tokens(data_iter):
    for text, _ in data_iter:
        yield tokenizer(text)
        
vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=['<unk>'])
vocab.set_default_index(vocab["<unk>"])

In [14]:
# 查看词典(字典形式)
vocab.get_stoi()
# {'dog': 3,'<unk>': 0, 'kidding': 5, 'cat': 1, 'ball': 4, 'The': 2, 'like': 6, 'mat': 7, 'on': 8, 'played': 9}

# 查看字典(列表形式)
vocab.get_itos()[:10]
# ['<unk>', '的', '我', '一下', '播放', '是', '吗', '给', '帮', '一个']

['<unk>', '的', '我', '一下', '播放', '是', '吗', '给', '帮', '一个']

In [16]:
# word to id
vocab(['我', '今天', '一下'])

[2, 41, 3]

In [17]:
# 找到text切词后对应的id
def text_pipeline(x):
    return vocab(tokenizer(x))

In [18]:
processed_text = torch.tensor(text_pipeline('今天我们在这里'), dtype=torch.int64)

processed_text

tensor([ 41, 262,  32, 663])

In [19]:
from torch.nn.utils.rnn import pad_sequence

In [52]:
from torch.utils.data import DataLoader
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 同一批数据的长度填充
# 90%的文本长度都在20个单词以内
def collate_batch(batch, max_len=20):
    label_list, text_list = [], []
    for (_text, _label) in batch:
        label_list.append(_label)
        
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        
#         pad 参数用于指定填充的数量，其格式为 [before, after]，其中 before 表示在张量的前面填充的值的数量，
# after 表示在张量的后面填充的值的数量。
# pad=[0, max_len] 表示在张量的前面不填充任何值，在张量的后面填充 max_len 个值。
# 这种填充的效果是，在张量的末尾添加了 max_len 个值为 0 的元素。
        processed_text = F.pad(processed_text, pad=[0, max_len], mode='constant', value=0)
        if len(processed_text) > max_len:
            processed_text = processed_text[:max_len]
        
        text_list.append(processed_text)
    
    label_list = torch.tensor(label_list, dtype=torch.int64)
    
    # 填充一个可变长度的张量列表。将长度较短的序列填充为和最长序列相同的长度。
    # 将填充后的序列列表堆叠成一个张量
    text_list = pad_sequence(text_list).T
    return label_list.to(device), text_list.to(device)

In [38]:
class BILSTM(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, label_size):
        super(BILSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = torch.nn.Embedding(vocab_size, embedding_dim)
        # self.embeddings.weight.data.copy_(torch.from_numpy(pretrained_w2v))
        
        self.lstm = torch.nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, bidirectional=True)
        self.hidden2label = torch.nn.Linear(hidden_dim*2, label_size)
        

    def forward(self, sentence):
        # torch.Size([16, 20])
        # print(sentence.shape)
        sentence = torch.transpose(sentence, 1, 0)
        
        # batch_size在后
        # torch.Size([20, 16])
        # print(sentence.shape)
        
        # torch.Size([20, 16, 100])
        x = self.embeddings(sentence)
        # print(x.shape)
        
        # 5 seqence length
        # 3 batch size
        # 10 input size
        # input = torch.randn(5, 3, 10)

        lstm_out, self.hidden = self.lstm(x)
        # torch.Size([20, 16, 128])
        # print(lstm_out.shape)
        y = self.hidden2label(lstm_out[-1,:,:])
        return y

In [35]:
a = torch.rand((2,3,1))
print(a.shape)

# transpose()一次只能在两个维度见进行转换，比如下面转置0维和2维
print(torch.transpose(torch.rand((2,3,1)), 0, 2).shape)

torch.Size([2, 3, 1])
torch.Size([1, 3, 2])


In [56]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    
    for idx, (label, text) in enumerate(dataloader):
        optimizer.zero_grad()  # 梯度清零
        
        predicted_label = model(text)
        
        loss = criterion(predicted_label, label)
        loss.backward()  # bp
        torch.nn.utils.clip_grad_norm(model.parameters(), 0.1)
        optimizer.step()  # 更新参数
        
        # argmax(1)：取列表每行的最大值
        # argmax(0): 取列表每列的最大值
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text) in enumerate(dataloader):
            predicted_label = model(text)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [57]:
num_class = len(lbl)
vocab_size = len(vocab)
emsize = 100
model = BILSTM(vocab_size, emsize, 64, num_class).to(device)

In [58]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset

# 超参数
EPOCHS = 40
LR = 2
BATCH_SIZE = 16

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 5.0, gamma=0.75)
total_accu = None

train_iter = coustom_data_iter(train_data[0].values[:], train_data[1].values[:])

# 将迭代器转化为 Dataset 类型
train_dataset = to_map_style_dataset(train_iter)

# 切分为训练集、验证集
num_train = int(len(train_dataset) * 0.75)
split_train_, split_valid_ = random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)


In [59]:
for (label, text) in train_dataloader:
    break
    
model(text).shape

torch.Size([16, 12])

In [60]:
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))

C:\Users\houhailun\AppData\Local\Temp\ipykernel_48468\1417882319.py:14: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  torch.nn.utils.clip_grad_norm(model.parameters(), 0.1)


| end of epoch   1 | time:  5.05s | valid accuracy    0.208 
| end of epoch   2 | time:  4.93s | valid accuracy    0.467 
| end of epoch   3 | time:  5.04s | valid accuracy    0.649 
| end of epoch   4 | time:  5.22s | valid accuracy    0.600 
| end of epoch   5 | time:  5.06s | valid accuracy    0.765 
| end of epoch   6 | time:  5.12s | valid accuracy    0.793 
| end of epoch   7 | time:  5.27s | valid accuracy    0.816 
| end of epoch   8 | time:  5.28s | valid accuracy    0.818 
| end of epoch   9 | time:  5.15s | valid accuracy    0.821 
| end of epoch  10 | time:  5.20s | valid accuracy    0.824 
| end of epoch  11 | time:  5.07s | valid accuracy    0.829 
| end of epoch  12 | time:  5.13s | valid accuracy    0.836 
| end of epoch  13 | time:  5.11s | valid accuracy    0.836 
| end of epoch  14 | time:  5.22s | valid accuracy    0.841 
| end of epoch  15 | time:  5.20s | valid accuracy    0.836 
| end of epoch  16 | time:  5.32s | valid accuracy    0.834 
| end of epoch  17 | tim

In [61]:
test_iter = coustom_data_iter(test_data[0].values[:], [0] * len(test_data))
test_dataset = to_map_style_dataset(test_iter)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=False, collate_fn=collate_batch)

In [62]:
def predict(dataloader):
    model.eval()

    test_pred = []
    with torch.no_grad():
        for idx, (label, text) in enumerate(dataloader):
            predicted_label = model(text).argmax(1)
            test_pred += list(predicted_label.cpu().numpy())
    return test_pred

In [63]:
test_pred = predict(test_dataloader)
test_pred = [lbl[x] for x in test_pred]

In [64]:
pd.DataFrame({
    'ID': range(1, len(test_pred) + 1),
    'Target': test_pred,
}).to_csv('nlp_submit.csv', index=None)